In [3]:
### to check the template mismatch after removing units
import spikeinterface.extractors as se
import  spikeinterface.postprocessing as spost
import spikeinterface.core as sc
import spikeinterface as si
import spikeinterface.curation as curation
from spikeinterface.extractors import BinaryRecordingExtractor
import spikeinterface.preprocessing as spre
import numpy as np
import matplotlib.pyplot as plt
import json
from functions import *
import warnings
warnings.simplefilter("ignore")
print(f"SpikeInterface version: {si.__version__}")

SpikeInterface version: 0.98.0.dev0


In [4]:
# set path
rec_name = "rec6"
base_folder = "/media/kang/Seagate_External/TJ_data_25/" + rec_name
phy_path = base_folder + "/kilosort2_5_results"
auto_curation_path = base_folder + "/sorted.npz"
# filtered_rec_path = phy_path + "/temp_wh.dat"   # use raw data, do filter and CMR
rec_path = base_folder + "/continuous.dat"
extract_path = base_folder + "/extract_waveforms"

# set parameters
fs = 30000
num_channel = 384

In [5]:
with open("exclude_channels_TJ_np.json", 'r') as j:
     exclude_channels = json.loads(j.read())[rec_name]
print(exclude_channels)
selected_units = select_units(auto_curation_path, exclude_channels)
print(selected_units)
phy_result = se.KiloSortSortingExtractor(phy_path)
print(phy_result)
auto_curated = phy_result.select_units(selected_units)
print(auto_curated)

[338, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384]
Number of units after auto curation:  225
[166, 170, 304, 302, 169, 303, 305, 183, 364, 164, 160, 297, 162, 298, 81, 262, 80, 179, 181, 367, 313, 178, 6, 5, 243, 32, 31, 29, 242, 240, 28, 241, 369, 318, 193, 371, 373, 236, 23, 24, 237, 223, 222, 7, 224, 168, 310, 175, 312, 334, 212, 333, 377, 295, 158, 136, 330, 331, 211, 332, 187, 314, 209, 329, 90, 87, 264, 255, 52, 140, 15, 229, 285, 287, 286, 294, 154, 362, 156, 384, 321, 375, 150, 360, 149, 260, 68, 220, 219, 2, 338, 205, 30, 4, 142, 235, 346, 22, 167, 163, 326, 14, 8, 340, 204, 10, 246, 245, 36, 130, 152, 293, 381, 261, 76, 155, 253, 49, 17, 195, 265, 92, 116, 275, 291, 144, 194, 105, 199, 123, 208, 42, 337, 316, 368, 19, 342, 344, 230, 231, 345, 232, 82, 382, 387, 129, 283, 44, 252, 69, 122, 315, 3, 324, 254, 47, 203, 272, 145, 39, 38, 309, 141, 358, 78, 112, 101, 115, 335, 325, 374, 383, 124, 125, 213, 336, 3

In [6]:
rec = BinaryRecordingExtractor(
        file_paths=rec_path,
        sampling_frequency=fs,
        num_chan=num_channel,
        dtype="int16",
        channel_ids=list(range(1, num_channel + 1))
    )
channel_locs = set_channel_loc_np()
rec.set_channel_locations(np.array(channel_locs))
rec

BinaryRecordingExtractor: 384 channels - 30.0kHz - 1 segments - 108,275,148 samples 
                          3,609.17s (1.00 hours) - int16 dtype - 77.44 GiB
  file_paths: ['/media/kang/Seagate_External/TJ_data_25/rec6/continuous.dat']

In [7]:
## filter and do cmr to the recording
rec_filt = spre.bandpass_filter(rec, freq_min=300, freq_max=6000)
rec_cmr = spre.common_reference(rec_filt, verbose=True)
rec_cmr

CommonReferenceRecording: 384 channels - 30.0kHz - 1 segments - 108,275,148 samples 
                          3,609.17s (1.00 hours) - int16 dtype - 77.44 GiB

In [8]:
import os
force_extract = False
job_kwargs = dict(n_jobs=8, progress_bar=True)

if os.path.isdir(extract_path) and not force_extract:
    we = sc.WaveformExtractor.load(folder=extract_path) 
    print(we)
else:
    we = sc.WaveformExtractor(rec_cmr, auto_curated, base_folder, allow_unfiltered=False)
    we.set_params(ms_before=3.0, ms_after=4.0)
    we.run_extract_waveforms( **job_kwargs)
    we.save(extract_path, overwrite=True)

WaveformExtractor: 384 channels - 225 units - 1 segments
  before:90 after:120 n_per_units:500


In [10]:
we._template_cache["average"].shape

(225, 210, 384)

In [11]:
best_channels = sc.get_template_extremum_channel(we)
print(best_channels)

{166: 274, 170: 274, 304: 278, 302: 270, 169: 274, 303: 278, 305: 274, 183: 286, 364: 270, 164: 270, 160: 270, 297: 270, 162: 270, 298: 270, 81: 142, 262: 142, 80: 142, 179: 282, 181: 282, 367: 284, 313: 284, 178: 278, 6: 16, 5: 16, 243: 36, 32: 36, 31: 36, 29: 34, 242: 36, 240: 34, 28: 34, 241: 34, 369: 298, 318: 300, 193: 301, 371: 300, 373: 301, 236: 30, 23: 30, 24: 30, 237: 30, 223: 18, 222: 18, 7: 18, 224: 18, 168: 275, 310: 278, 175: 278, 312: 282, 334: 334, 212: 334, 333: 334, 377: 334, 295: 266, 158: 265, 136: 235, 330: 330, 331: 330, 211: 332, 332: 332, 187: 288, 314: 286, 209: 327, 329: 327, 90: 158, 87: 158, 264: 158, 255: 62, 52: 62, 140: 242, 15: 24, 229: 26, 285: 230, 287: 229, 286: 230, 294: 261, 154: 261, 362: 263, 156: 263, 384: 314, 321: 314, 375: 314, 150: 254, 360: 256, 149: 253, 260: 110, 68: 110, 220: 13, 219: 10, 2: 10, 338: 10, 205: 321, 30: 35, 4: 14, 142: 245, 235: 29, 346: 29, 22: 31, 167: 271, 163: 270, 326: 322, 14: 24, 8: 22, 340: 20, 204: 320, 10: 19, 246

In [12]:
# from spikeinterface.core.template_tools import get_template_amplitudes
redundant_units = curation.find_redundant_units(we.sorting)
print(redundant_units)

# peak_values = get_template_amplitudes(we, peak_sign="neg", mode="at_index")
# peak_values = {unit_id: np.max(np.abs(values)) for unit_id, values in peak_values.items()}
# remove_unit_ids = []
curated_redundant, redundant_unit_pairs, remove_unit_ids = \
    curation.remove_redundant_units(we, align=False, remove_strategy="highest_amplitude", extra_outputs=True)
# for u1, u2 in redundant_unit_pairs:
#     if np.abs(peak_values[u1]) < np.abs(peak_values[u2]):
#         remove_unit_ids.append(u1)
#     else:
#         remove_unit_ids.append(u2)

print(len(redundant_unit_pairs), redundant_unit_pairs)
print(remove_unit_ids)
print(curated_redundant)
we.sorting = curated_redundant

[[262, 81], [32, 243], [163, 364], [14, 15], [19, 235], [82, 81], [82, 262], [177, 179], [244, 245], [339, 10], [176, 367], [280, 123], [171, 166], [171, 386]]
14 [[262, 81], [32, 243], [163, 364], [14, 15], [19, 235], [82, 81], [82, 262], [177, 179], [244, 245], [339, 10], [176, 367], [280, 123], [171, 166], [171, 386]]
[262, 243, 163, 14, 19, 82, 262, 177, 245, 339, 176, 280, 171, 386]
UnitsSelectionSorting: 212 units - 1 segments - 30.0kHz


In [13]:
we._template_cache["average"].shape

(225, 210, 384)

In [14]:
best_channels_re = sc.get_template_extremum_channel(we)
diff_counter = 0
for k, v in best_channels_re.items():
    if v != best_channels[k]:
        diff_counter += 1
diff_counter

177

In [15]:
# manually remove templates
import copy
original_ids = auto_curated.get_unit_ids().tolist()
indices_to_remove = [original_ids.index(id) for id in remove_unit_ids]
original_templates = copy.deepcopy(we._template_cache['average'])
we._template_cache["average"] = np.delete(original_templates, indices_to_remove, axis=0)
print(we._template_cache["average"].shape)

(212, 210, 384)


In [16]:
best_channels_mn = sc.get_template_extremum_channel(we)
diff_counter = 0
for k, v in best_channels_mn.items():
    if v != best_channels[k]:
        diff_counter += 1
diff_counter

0